In [20]:
# librairies
from sklearn.mixture import GaussianMixture
from sklearn.decomposition import PCA
from skimage.transform import resize
import pandas as pd
import numpy as np
import pickle

In [21]:
# read extracted features from pickle file
def read_extracted_features(extracted_features_save_adr):
    with open(extracted_features_save_adr, 'rb') as input_file:
        return pickle.load(input_file)

In [22]:
extracted_features_save_adr = "../extracted_features.pickle"
features = read_extracted_features(extracted_features_save_adr)
features = dict(features)
features
len(features)

170

In [4]:
new_features = {}

In [5]:
for patient in list(features.keys()):
  featt = np.vstack(list(features[patient].values()))
  pca = PCA(n_components=10, svd_solver='full', whiten=True)
  feat1=pca.fit_transform(featt)
  new_features.update({patient:feat1})

In [6]:
len(new_features)
patient_1 = list(new_features.keys())[0]
len(new_features[patient_1])

129

In [23]:
concat_features = {}
for patient in new_features.keys():
    gmm = GaussianMixture(n_components=len(new_features[patient]))
    gmm.fit(new_features[patient])
    probs = gmm.predict_proba(new_features[patient])
    proba_resized = resize(probs, (len(new_features[patient]), 10), preserve_range=True)
    concat_features.update({patient: (proba_resized*new_features[patient]).sum(axis=0)})

TypeError: float() argument must be a string or a number, not 'dict'

In [8]:
len(concat_features)
patient_1 = list(concat_features.keys())[0]
len(concat_features[patient_1])
concat_features

{'13901': array([ 0.18423706, -0.27321938,  0.00268916,  0.14110579, -0.19562355,
        -0.10926075, -0.0693151 ,  0.0597403 , -0.23965298, -0.27007259]),
 '13902': array([ 0.06813593,  0.29358454,  0.24625958,  0.07632135,  0.06882143,
        -0.01509653, -0.0246769 ,  0.07690753, -0.33974016, -0.12732443]),
 '13903': array([-0.13889205, -0.07339546,  0.14472891,  0.17257591,  0.09263924,
        -0.05244348, -0.11442381, -0.23537733, -0.1478019 , -0.3314971 ]),
 '13904': array([ 0.43030101,  0.03919099, -0.26820742,  0.19315046, -0.35282588,
        -0.09755991,  0.08496533,  0.23596658,  0.45989276, -0.21403637]),
 '13905': array([ 0.45938082,  0.4684838 , -0.23472536, -0.76254238,  0.50654693,
        -0.58961952,  0.04224634,  0.13500034, -0.28688927,  0.32719063]),
 '13906': array([ 0.21678754, -0.07190553, -0.27644073,  0.16392085,  0.32345546,
         0.43668112, -0.32306734,  0.1186051 , -0.21779089, -0.39883062]),
 '13908': array([-0.07853231, -0.16326265,  0.25254539,  0

In [15]:
# Convert the dictionary to a csv file where each row is a patient and each column is a feature using pandas
df = pd.DataFrame.from_dict(concat_features, orient='index')
df.to_csv('../CSV/features_concatenated_10.csv', index=True, header=True)

In [19]:
# Merge the extracted features with the clinical data on the patient ID
clinical_data = pd.read_csv('../CSV/clinical_data_with_no_missing_values.csv')
clinical_data = clinical_data.set_index('patient_id')
features = pd.read_csv('../CSV/features_concatenated_10.csv')
features = features.set_index('Unnamed: 0')
features = clinical_data.join(features)
features.to_csv('../CSV/features_concatenated_with_clinical_data_10.csv', index=True, header=True)